<a href="https://colab.research.google.com/github/ficle-fr/pix2pix_change_style/blob/colabs/colabs/train_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>